# Lista 05 - Regresão

>Yanna Torres Gonçalves
>
>Matrícula: 587299
>
>Mestrado em Ciências da Computação


## Parte 01: limpeza e pré-processamento dos dados

In [31]:
!pip install faiss-gpu-cu12

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 8.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip install scikit-learn fuzzywuzzy python-Levenshtein spacy nltk sentence_transformers emoji unidecode

In [1]:
!python -m spacy download pt_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.8.0/pt_core_news_sm-3.8.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
import pandas as pd
import numpy as np
import re

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
import spacy
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### a) Ler o dataset fakeTelegram.BR_2022.csv

Utilizamos o mesmo dataset da lista 01, acessando pelo mesmo link.

[https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv](https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv)
   

In [3]:
data_url = "https://media.githubusercontent.com/media/yanna-torres/CKP9011-ciencia-de-dados/refs/heads/lista-01/data/fakeTelegram.BR_2022.csv"

df = pd.read_csv(data_url)

In [4]:
df.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,557586.0,444157.000000,167238.000000,5.575860e+05
mean,5.0,0.017330,0.312245,4.450617e+05
std,0.0,0.464165,0.293699,4.860211e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078454,2.127500e+04
50%,5.0,0.000000,0.197577,1.210935e+05
75%,5.0,0.318200,0.490351,9.726045e+05
max,5.0,0.999200,1.000000,1.516436e+06


### b) Remova os trava-zaps, as linhas repetidas e textos com menos de 5 palavras

#### Removendo os trava-zaps

In [5]:
def has_trava_zap(message):
    if not isinstance(message, str):
        return False  # Garante que só analisa strings

    # Regra 1: muito longo
    if len(message) > 10000:
        return True

    # Regra 2: caracteres invisíveis
    invisible = ''.join(chr(c) for c in range(0x200B, 0x200F + 1)) + '\uFEFF'
    if sum(message.count(c) for c in invisible) > 100:
        return True

    # Regra 3: emojis repetidos
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                              "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
    emojis = emoji_pattern.findall(message)
    if any(len(e) > 20 for e in emojis):
        return True

    # Regra 4: baixa diversidade
    unique = set(message)
    if len(unique) < 10 and len(message) > 500:
        return True

    return False

In [6]:
df['trava_zap'] = df['text_content_anonymous'].apply(has_trava_zap)

In [7]:
df_clean = df[df['trava_zap'] == False]

In [8]:
df_clean.drop(columns=['trava_zap'], inplace=True)

/tmp/ipykernel_49870/1420576694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(columns=['trava_zap'], inplace=True)


In [9]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,556305.0,442876.000000,166416.000000,5.563050e+05
mean,5.0,0.017676,0.311496,4.448514e+05
std,0.0,0.463547,0.293146,4.859544e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078334,2.125100e+04
50%,5.0,0.000000,0.196892,1.208690e+05
75%,5.0,0.318200,0.488192,9.724220e+05
max,5.0,0.999200,1.000000,1.516436e+06


#### Removendo as linhas repetidas

In [10]:
df_clean = df_clean.drop_duplicates()

In [11]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,556305.0,442876.000000,166416.000000,5.563050e+05
mean,5.0,0.017676,0.311496,4.448514e+05
std,0.0,0.463547,0.293146,4.859544e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078334,2.125100e+04
50%,5.0,0.000000,0.196892,1.208690e+05
75%,5.0,0.318200,0.488192,9.724220e+05
max,5.0,0.999200,1.000000,1.516436e+06


#### Removendo textos com menos de 5 palavras

In [12]:
df_clean = df_clean[df_clean['text_content_anonymous'].str.split().str.len() >= 5]

In [13]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,338045.0,337998.000000,166402.000000,3.380450e+05
mean,5.0,0.022497,0.311514,4.214875e+05
std,0.0,0.529214,0.293150,4.842346e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.381800,0.078340,1.988800e+04
50%,5.0,0.000000,0.196925,8.473500e+04
75%,5.0,0.505300,0.488286,9.726970e+05
max,5.0,0.999200,1.000000,1.516436e+06


### c) Agrupe as linhas com postagens iguais ou semelhantes

In [14]:
df_clean.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'message_type',
       'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [15]:
import emoji
import unidecode

def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = emoji.replace_emoji(text, "")
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = unidecode.unidecode(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [16]:
df_clean['text_normalized'] = df_clean['text_content_anonymous'].apply(normalize_text)

In [17]:
len(df_clean)

338045

In [18]:
df_clean = df_clean[df_clean['text_normalized'].notnull()]
df_clean = df_clean[df_clean['text_normalized'] != ""]

In [19]:
len(df_clean)

337529

In [20]:
# Remove exact duplicates and count occurrences as "shares"
df_grouped = (
    df_clean.groupby('text_normalized', as_index=False)
      .agg({
            'date_message': 'min',
            'id_member_anonymous': 'first',
            **{col: 'first' for col in df_clean.columns if col != 'text_normalized' and col != 'date_message'}
      })
)

In [21]:
share_counts = df_clean['text_content_anonymous'].value_counts().to_dict()
df_grouped['shares'] = df_grouped['text_content_anonymous'].map(share_counts)

In [22]:
len(df_grouped)

214429

In [23]:
df_grouped.head(5)

,text_normalized,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,shares
0,0 800nana alaia azzedineparis france011 331jea...,2022-10-02 02:53:42,None,6e4192de8ce3464c5263b0a937e53529,None,None,None,False,False,",,""0 800"",""N/A"",""N/A"",\n\n""Alaia, Azzedine"",""P...",5,2022-10-03 04:54:51.879906,0.0000,0.003349,19900,Texto,telegram,None,None,1
1,0 a 10 vai ter 2o turno o que vocs acham,2022-10-03 00:11:39,None,b11f2df64ac19aad47a50accf32052d6,None,None,None,False,False,"0 a 10 vai ter 2o turno, o que vocês acham ?",5,2022-10-03 05:18:48.986706,0.0000,NaN,154118,Texto,telegram,None,None,1
2,0 cabea de ovo no respeita mais nada eles e do...,2022-10-28 02:47:19,None,e56ec342fc599ebb4ed89655eb6f03aa,None,None,None,False,False,0 cabeça de ovo não respeita mais nada eles ...,5,2022-10-28 02:47:24.04081,0.0258,NaN,540263,Texto,telegram,None,None,1
3,0 g0lpe t a t3 tem ch0que derealidade all ads ...,2022-09-29 03:44:33,None,5b10d7739171149be6d9961e3350c071,3327c1bedc6235177f5d48885d87b39d.jpg,url,https://www.youtube.com/watch?v=gwYdiCsaBoM,True,True,'0 G0LPE TÁ AÍ! - T$3 TEM CH0QUE DEREALIDADE'\...,5,2022-10-03 04:07:17.895291,-0.6352,NaN,1052465,Url,telegram,None,3327c1bedc6235177f5d48885d87b39d,1
4,0 mrs jenniferbrooksfx,2022-10-18 16:11:18,32d1832a2df01fd4a2a59b3ef8642c58,5b10d7739171149be6d9961e3350c071,None,None,https://t.me/+8neVAtvN9_4xODM0,False,True,https://t.me/+8neVAtvN9_4xODM0\n𝐈 𝐡𝐚𝐯𝐞 𝐭𝐫𝐢𝐞𝐝 𝐬...,5,2022-10-18 16:11:32.552063,0.0000,0.111523,1147539,Texto,telegram,None,None,2


In [24]:
from sentence_transformers import SentenceTransformer
import torch

# Load a multilingual model (Portuguese-friendly) and move to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

/opt/conda/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-06-10 23:54:12.108650: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 23:54:12.108717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 23:54:12.109909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
# Get embeddings (batch-processed on GPU)
texts = df_grouped['text_normalized'].tolist()
print("Encoding texts into embeddings...")
embeddings = model.encode(texts, batch_size=128, show_progress_bar=True, device=device)
df_grouped['embedding'] = embeddings.tolist()

Encoding texts into embeddings...


Batches:   0%|          | 0/1676 [00:00<?, ?it/s]

In [28]:
df_grouped = df_grouped.reset_index(drop=True)
embeddings_array = np.array(df_grouped['embedding'].tolist())

In [33]:
import faiss

faiss.normalize_L2(embeddings)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

In [35]:
from collections import defaultdict

similarity_threshold = 0.85
k = 10

clusters = defaultdict(list)
removed_indices = set()

for i in tqdm(range(len(df_grouped))):
    if i in removed_indices:
        continue
    vec = embeddings[i].reshape(1, -1)
    D, I = index.search(vec, k)
    for sim, j in zip(D[0][1:], I[0][1:]):
        if j == -1 or j == i or j in removed_indices:
            continue
        if sim >= similarity_threshold:
            clusters[i].append(j)
            removed_indices.add(j)

100%|██████████| 214429/214429 [2:05:13<00:00, 28.54it/s]  


In [37]:
results = []
used = set()

for base_idx, sim_indices in tqdm(clusters.items()):
    if base_idx in used:
        continue
    total_count = df_grouped.iloc[base_idx]['shares']
    used.add(base_idx)
    for sim_idx in sim_indices:
        total_count += df_grouped.iloc[sim_idx]['shares']
        used.add(sim_idx)
    row = df_grouped.iloc[base_idx].copy()
    row['shares'] = total_count
    results.append(row)

# Add remaining unmatched
remaining = df_grouped.loc[~df_grouped.index.isin(used)].copy()
final_df = pd.concat([pd.DataFrame(results), remaining], ignore_index=True)

100%|██████████| 18189/18189 [00:05<00:00, 3425.61it/s]


In [38]:
len(final_df)

178987

In [41]:
final_df.head()

,text_normalized,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,shares,embedding
0,0 mrs jenniferbrooksfx,2022-10-18 16:11:18,32d1832a2df01fd4a2a59b3ef8642c58,5b10d7739171149be6d9961e3350c071,None,None,https://t.me/+8neVAtvN9_4xODM0,False,True,https://t.me/+8neVAtvN9_4xODM0\n𝐈 𝐡𝐚𝐯𝐞 𝐭𝐫𝐢𝐞𝐝 𝐬...,...,2022-10-18 16:11:32.552063,0.0,0.111523,1147539,Texto,telegram,None,None,4,"[-0.04452281817793846, 0.06801902502775192, 0...."
1,01112022 0800 siapacfico fechado spasx 165 697...,2022-11-01 11:01:54,None,2ff252ad4422e11a6a8abfaa747abb55,None,None,youtube.com/AlexEconomia,False,True,▪️ 01/11/2022 - 08:00 \n\nÁsia-Pacífico (fecha...,...,2022-11-01 11:02:13.43606,0.0,0.065628,52387,Texto,telegram,None,None,10,"[-0.010921835899353027, -0.024950524792075157,..."
2,02102022 tse governosp s 18h 59m 00s,2022-10-02 22:05:53,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,a9dff1444485730d391ca9720108cdb7.jpg,image/jpg,None,True,False,*02.10.2022 TSE - Governo-SP às 18h 59m 00s*,...,2022-10-03 05:13:14.67546,0.0,NaN,43651,Imagem,telegram,None,a9dff1444485730d391ca9720108cdb7,10,"[-0.0907742977142334, 0.04765523225069046, 0.0..."
3,02102022 tse governosp s 20h 50m 48s,2022-10-02 23:57:28,None,ef6bcfbd08be365e9b208c22f1d6ad36,9259e029efd410636242ea00ca5d0017.jpg,image/jpg,None,True,False,02.10.2022 TSE - Governo-SP às 20h 50m 48s,...,2022-10-03 05:18:11.945647,0.0,NaN,43667,Imagem,telegram,None,9259e029efd410636242ea00ca5d0017,2,"[-0.09553123265504837, 0.06292906403541565, 0...."
4,02102022 tse presidente s 18h 55m 31s,2022-10-02 21:59:55,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,b18ed539a09a227d1fa8de9ad82d7214.jpg,image/jpg,None,True,False,02.10.2022 TSE - Presidente - às 18h 55m 31s,...,2022-10-03 05:13:05.88844,0.0,NaN,43650,Imagem,telegram,None,b18ed539a09a227d1fa8de9ad82d7214,10,"[-0.1111832931637764, 0.04023043066263199, 0.0..."


In [44]:
final_df.drop(columns=['embedding']).to_csv("final.csv")

In [47]:
final_df[['text_content_anonymous', 'shares']].tail()

,text_content_anonymous,shares
178982,‘Zum Zum Zum’ em brasólia…\n\nA bem da paz soc...,1
178983,ZUMBI NÃO ESCOLHE LUGAR PARA MORDER. O OBJETIV...,1
178984,ZUMBIS MODIFICADOS NO DNA PELA VACINA.. NÃO SE...,2
178985,Zuverlässige und vertrauenswürdige Zahlungssei...,1
178986,Zyklon B era pra matar piolhos. A manipulação ...,1


---

### d) Colunas Auxiliares

Para facilitar a recuperação de dados em alguns casos, vamos criar colunas auxiliares.

In [50]:
stop_words = set(stopwords.words('portuguese', 'english'))

tqdm.pandas()

def clean_text(text):
    if not isinstance(text, str):
        return ''
    text_no_punct = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text_no_punct)
    filtered = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered)

final_df['text_no_stopwords'] = final_df['text_content_anonymous'].progress_apply(clean_text)

100%|██████████| 178987/178987 [00:26<00:00, 6824.98it/s]


In [51]:
final_df['word_count'] = final_df['text_content_anonymous'].progress_apply(lambda x: len(x.split()) if pd.notnull(x) else 0)
final_df['character_count'] = final_df['text_content_anonymous'].progress_apply(lambda x: len(x) if pd.notnull(x) else 0)

100%|██████████| 178987/178987 [00:00<00:00, 573190.02it/s]


In [53]:
def classify_misinformation(score):
    if score >= 0.66:
        return 'Misinformation'
    elif score >= 0.33:
        return 'Neutral'
    else:
        return 'Non-misinformation'

# Criar coluna de categoria
final_df['misinformation_category'] = final_df['score_misinformation'].progress_apply(classify_misinformation)

100%|██████████| 178987/178987 [00:00<00:00, 948506.59it/s]


In [54]:
final_df['sentiment'] = final_df['score_sentiment'].progress_apply(lambda x: "Positive" if x >= 0.05 else ("Negative" if x <= -0.05 else "Neutral"))

100%|██████████| 178987/178987 [00:00<00:00, 940014.79it/s]


In [55]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178987 entries, 0 to 178986
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   text_normalized          178987 non-null  object 
 1   date_message             178987 non-null  object 
 2   id_member_anonymous      78876 non-null   object 
 3   id_group_anonymous       178987 non-null  object 
 4   media                    45782 non-null   object 
 5   media_type               45782 non-null   object 
 6   media_url                33158 non-null   object 
 7   has_media                178987 non-null  bool   
 8   has_media_url            178987 non-null  bool   
 9   text_content_anonymous   178987 non-null  object 
 10  dataset_info_id          178987 non-null  int64  
 11  date_system              178987 non-null  object 
 12  score_sentiment          178958 non-null  float64
 13  score_misinformation     82812 non-null   float64
 14  id_m

In [56]:
final_df.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message,shares,word_count,character_count
count,178987.0,178958.000000,82812.000000,1.789870e+05,178987.000000,178987.000000,178987.000000
mean,5.0,-0.076506,0.314753,4.264944e+05,1.814931,34.992011,219.425254
std,0.0,0.498746,0.271386,4.877426e+05,41.451526,57.755891,359.596507
min,5.0,-1.000000,0.000003,2.000000e+00,1.000000,5.000000,12.000000
25%,5.0,-0.476700,0.100610,2.399700e+04,1.000000,13.000000,81.000000
50%,5.0,0.000000,0.220655,8.619300e+04,1.000000,19.000000,123.000000
75%,5.0,0.273200,0.471673,9.679240e+05,1.000000,33.000000,214.000000
max,5.0,0.999200,1.000000,1.516427e+06,17422.000000,993.000000,4096.000000


In [57]:
final_df.drop(columns=['embedding']).to_csv("final_with_columns.csv")

## Parte 02: Modelo Preditivo

Utilizando os dados referente a postagens no Telegram, crie um modelo preditivo
(regressor) para, dado os dados de uma postagem, prever a quantidade de
compartilhamentos dessa mensagem, o que é denominado potencial de “viralização”.

### Pré-processamento

Removendo mensagens com caracteres 

In [62]:
final_df.columns

Index(['text_normalized', 'date_message', 'id_member_anonymous',
       'id_group_anonymous', 'media', 'media_type', 'media_url', 'has_media',
       'has_media_url', 'text_content_anonymous', 'dataset_info_id',
       'date_system', 'score_sentiment', 'score_misinformation', 'id_message',
       'message_type', 'messenger', 'media_name', 'media_md5', 'shares',
       'embedding', 'text_no_stopwords', 'word_count', 'character_count',
       'misinformation_category', 'sentiment'],
      dtype='object')

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove spans e tags
    text = re.sub(r"<[^>]+>", "", text)
    # Remove caracteres não latinos
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    return text

In [69]:
final_df['text_clean'] = final_df['text_no_stopwords'].progress_apply(clean_text)

100%|██████████| 178987/178987 [00:20<00:00, 8632.77it/s] 


In [70]:
final_df = final_df.dropna(subset=['shares', 'text_clean'])

In [71]:
len(final_df)

178987

In [73]:
def is_latin_only(text):
    if not isinstance(text, str):
        return False
    # This regex matches text containing **any** non-Latin characters
    return not re.search(r'[^\u0000-\u007F]', text)

In [74]:
final_df = final_df[final_df['text_no_stopwords'].apply(is_latin_only)]

In [79]:
len(final_df)

34108

In [81]:
final_df[["shares", "text_content_anonymous"]].sort_values(by='shares', ascending=False).head(10)

,shares,text_content_anonymous
162719,17422,This community was blocked in Brazil following...
7870,375,Foto de Maria Teresa D Valente
2086,108,Arquivo de Maria Emilia Gadelha Serr
554,79,"A FRAUDE JÁ ESTAVA FEITA, POR ISSO O MOTIVO DE..."
9788,72,meu grande investidor está testemunhando como ...
6332,59,ENTRE PESSOAL AQUI PETISTA NÃO TEM VEZ\n\nhttp...
1768,57,ANTES DE VOTAR VEJA ISSO.pdf
13616,54,SE JÁ TIROU UMA FOTO COM MAIS DE UM COMPROVANT...
1860,53,https://enqueteeleicoes2022.com.br/\nAO VIVO: ...
1305,48,Aguardem bando de PTistas do inferno. A hora d...


In [82]:
# Mensagem do Governo
final_df = final_df.drop(index=162719)

In [83]:
final_df[["shares", "text_content_anonymous"]].sort_values(by='shares', ascending=False).head(10)

,shares,text_content_anonymous
7870,375,Foto de Maria Teresa D Valente
2086,108,Arquivo de Maria Emilia Gadelha Serr
554,79,"A FRAUDE JÁ ESTAVA FEITA, POR ISSO O MOTIVO DE..."
9788,72,meu grande investidor está testemunhando como ...
6332,59,ENTRE PESSOAL AQUI PETISTA NÃO TEM VEZ\n\nhttp...
1768,57,ANTES DE VOTAR VEJA ISSO.pdf
13616,54,SE JÁ TIROU UMA FOTO COM MAIS DE UM COMPROVANT...
1860,53,https://enqueteeleicoes2022.com.br/\nAO VIVO: ...
1305,48,Aguardem bando de PTistas do inferno. A hora d...
3707,47,bom dia a todos os meus assinantes para aquele...


In [84]:
final_df.drop(columns=['embedding']).to_csv("final_with_columns_clean.csv")

In [90]:
final_df.head()

,text_normalized,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,...,media_name,media_md5,shares,embedding,text_no_stopwords,word_count,character_count,misinformation_category,sentiment,text_clean
2,02102022 tse governosp s 18h 59m 00s,2022-10-02 22:05:53,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,a9dff1444485730d391ca9720108cdb7.jpg,image/jpg,None,1,0,*02.10.2022 TSE - Governo-SP às 18h 59m 00s*,...,None,a9dff1444485730d391ca9720108cdb7,10,"[-0.0907742977142334, 0.04765523225069046, 0.0...",02102022 TSE GovernoSP 18h 59m 00s,8,44,Non-misinformation,Neutral,02102022 TSE GovernoSP 18h 59m 00s
3,02102022 tse governosp s 20h 50m 48s,2022-10-02 23:57:28,None,ef6bcfbd08be365e9b208c22f1d6ad36,9259e029efd410636242ea00ca5d0017.jpg,image/jpg,None,1,0,02.10.2022 TSE - Governo-SP às 20h 50m 48s,...,None,9259e029efd410636242ea00ca5d0017,2,"[-0.09553123265504837, 0.06292906403541565, 0....",02102022 TSE GovernoSP 20h 50m 48s,8,42,Non-misinformation,Neutral,02102022 TSE GovernoSP 20h 50m 48s
4,02102022 tse presidente s 18h 55m 31s,2022-10-02 21:59:55,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,b18ed539a09a227d1fa8de9ad82d7214.jpg,image/jpg,None,1,0,02.10.2022 TSE - Presidente - às 18h 55m 31s,...,None,b18ed539a09a227d1fa8de9ad82d7214,10,"[-0.1111832931637764, 0.04023043066263199, 0.0...",02102022 TSE Presidente 18h 55m 31s,9,44,Non-misinformation,Neutral,02102022 TSE Presidente 18h 55m 31s
5,02102022 tse presidente s 20h 02m 45s,2022-10-02 23:17:15,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,9df52fce3ae65d3fe701e5d1d85f2cc5.jpg,image/jpg,None,1,0,*02.10.2022 TSE - Presidente às 20h 02m 45s*,...,None,9df52fce3ae65d3fe701e5d1d85f2cc5,3,"[-0.11566051095724106, 0.041493937373161316, 0...",02102022 TSE Presidente 20h 02m 45s,8,44,Non-misinformation,Neutral,02102022 TSE Presidente 20h 02m 45s
6,02102022 tse presidente s 22h 21m 36s,2022-10-03 01:26:41,56b8359fd127312651b80b8ed8030085,ef6bcfbd08be365e9b208c22f1d6ad36,538d5e2d9c6f1cdbb889bfc7f5f6c857.jpg,image/jpg,None,1,0,02.10.2022 TSE - Presidente às 22h 21m 36s,...,None,538d5e2d9c6f1cdbb889bfc7f5f6c857,2,"[-0.08145476877689362, 0.03734157234430313, 0....",02102022 TSE Presidente 22h 21m 36s,8,42,Non-misinformation,Neutral,02102022 TSE Presidente 22h 21m 36s


### 